# <span style="background-color:white;color:red;"> ⌛⌛ EN PROGRESO. NO ES ÚTIL⌛⌛</span>

## Bloque I
Descomprimir el csv en formato gz

In [4]:
"""  
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
from config_bd import *
import requests
import sys
import gzip
import shutil

DATA_DIR = os.path.join('data')
# DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


""" Comprobamos que existe el directorio de datos """

if not os.path.exists(DATA_DIR):
    print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
    sys.exit()

""" Descomprimimos el archivo de calendario """

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)


gz_file_path = os.path.join(DATA_DIR, 'calendar.csv.gz')
output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

if os.path.exists(gz_file_path):
    try:
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
    except Exception as e:
        print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
        sys.exit()
else:
    print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
    sys.exit()


Archivo data\calendar.csv.gz descomprimido como data\TEMP\calendar.csv


## Bloque II
Carga del csv en pandas. Cambios de tipos e imputación

Estos son lo pasos seguidos:

<ul>
<li>Elimino el simbolo de la moneda y lo convierto a float.</li>
<li>Convierto la disponibilidad de string a bool (0,1)</li>
<li>Convierto las max y min nigths a enteros. Imputo los faltantas con las medianas de cada una.</li>
<li>Elimino el precio ajustado.</li>
<li>Renombro el listing_id a property_id</li>
</ul>

In [5]:
df = pd.read_csv(output_file_path, low_memory=False)

In [6]:
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['price'] = df['price'].fillna(0)
df['available'] = df['available'].replace({'t': 1, 'f': 0})
df['minimum_nights'] = df['minimum_nights'].fillna(df['minimum_nights'].median()).astype(int)
df['maximum_nights'] = df['maximum_nights'].fillna(df['maximum_nights'].median()).astype(int)
df = df.drop(columns=['adjusted_price'])
df = df.rename(columns={'listing_id': 'property_id'})
# df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

C:\Users\dsantamaria\AppData\Local\Temp\ipykernel_12100\1973130110.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['available'] = df['available'].replace({'t': 1, 'f': 0})


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
property_id,9236806.0,6.438998e+17,5.357870e+17,21853.0,36760070.0,8.115293e+17,1.142566e+18,1.369179e+18
available,9236806.0,4.376801e-01,4.961011e-01,0.0,0.0,0.000000e+00,1.000000e+00,1.000000e+00
price,9236806.0,2.116178e+02,6.396718e+02,8.0,55.0,9.200000e+01,1.600000e+02,2.100000e+04
minimum_nights,9236806.0,1.016236e+01,3.081177e+01,1.0,1.0,2.000000e+00,6.000000e+00,1.125000e+03
maximum_nights,9236806.0,6.142410e+02,4.475243e+02,1.0,364.0,3.650000e+02,1.125000e+03,1.111100e+04


In [8]:
df.sample(10)

,property_id,date,available,price,minimum_nights,maximum_nights
2768671,42986180,2025-04-28,1,500.0,4,1125
9081265,1357979122503430399,2026-02-20,0,160.0,4,365
8376057,1296632865931017797,2025-09-27,0,70.0,2,365
1057933,19804684,2026-01-09,1,50.0,2,1125
3907963,654080915562183035,2025-04-08,0,146.0,30,1125
5343093,930822032761074701,2025-04-04,1,45.0,3,365
3414716,53397401,2025-05-15,1,80.0,5,1125
220987,4604059,2025-09-12,0,35.0,2,1125
750532,16204754,2025-04-20,1,85.0,24,1125
907785,18305543,2025-10-01,0,92.0,2,1125


In [9]:
df.describe()

,property_id,available,price,minimum_nights,maximum_nights
count,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06,9.236806e+06
mean,6.438998e+17,4.376801e-01,2.116178e+02,1.016236e+01,6.142410e+02
std,5.357870e+17,4.961011e-01,6.396718e+02,3.081177e+01,4.475243e+02
min,2.185300e+04,0.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00
25%,3.676007e+07,0.000000e+00,5.500000e+01,1.000000e+00,3.640000e+02
50%,8.115293e+17,0.000000e+00,9.200000e+01,2.000000e+00,3.650000e+02
75%,1.142566e+18,1.000000e+00,1.600000e+02,6.000000e+00,1.125000e+03
max,1.369179e+18,1.000000e+00,2.100000e+04,1.125000e+03,1.111100e+04


## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla** <code>01_CREATE TABLE bnb_calendar.sql</code>

In [10]:
from config_bd import get_connection

In [11]:
df.to_csv(os.path.join(TEMP_DIR,'transform_calendar.csv'), index=False)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9236806 entries, 0 to 9236805
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   property_id     int64  
 1   date            object 
 2   available       int64  
 3   price           float64
 4   minimum_nights  int64  
 5   maximum_nights  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 422.8+ MB


In [41]:
from tqdm import tqdm
# from time import sleep

with get_connection() as conn:
    cursor = conn.cursor()
    cursor.fast_executemany = True 

    CHUNKSIZE = 100000
    for chunk in tqdm(
        pd.read_csv(os.path.join(TEMP_DIR, 'transform_calendar.csv'), chunksize=CHUNKSIZE),
        unit_scale=CHUNKSIZE, unit=" filas"
    ):

        insert_sql = """
        INSERT INTO dbo.bnb_calendar
            (property_id, [date], available, price,
            minimum_nights, maximum_nights)
        VALUES (?,?,?,?,?,?)
        """
        cursor.executemany(insert_sql, chunk.values.tolist())
        conn.commit()        




0 filas [00:00, ? filas/s]

9300000 filas [08:31, 18174.81 filas/s]


Elimino el fichero para no dejar duplicado el csv

In [42]:
os.remove(os.path.join(TEMP_DIR,'transform_calendar.csv'))